In [ ]:
##Request the data and convert to a csv file

In [ ]:
#import the following libraries
import requests
import json
import pandas as pd
import os
import os.path

In [ ]:
#you will need an API key, which you can get through https://www.census.gov/data/developers.html
api_key_input = input('Enter your API key: ')
census_api_key = api_key_input

In [ ]:
#states dictionary

states = {'AL':'01', 'AK':'02', 'AZ':'04', 'AR':'05', 'CA':'06', 'CO':'08', 'CT':'09', 'DE':'10', 'DC':'11', 'FL':'12', 'GA':'13', 'HI':'15', 
            'ID':'16', 'IL':'17', 'IN':'18', 'IA':'19', 'KS':'20', 'KY':'21', 'LA':'22', 'ME':'23', 'MD':'24', 'MA':'25', 'MI':'26', 'MN':'27', 
            'MS':'28', 'MO':'29', 'MT':'30', 'NE':'31', 'NV':'32', 'NH':'33', 'NJ':'34', 'NM':'35', 'NY':'36', 'NC':'37', 'ND':'38', 'OH':'39', 
            'OK':'40', 'OR':'41', 'PA':'42', 'RI':'44', 'SC':'45', 'SD':'46', 'TN':'47', 'TX':'48', 'UT':'49', 'VT':'50', 'VA':'51', 'WA':'53', 
            'WV':'54', 'WI':'55', 'WY':'56', 'AS':'60', 'GU':'66', 'MP':'69', 'PR':'72', 'VI':'78'}
            
while True:
    state_input = input('Enter a valid two letter abbreviation for the state or territory of your choice: ')
    if state_input not in states:
        print('Abbreviation is invalid')
        continue
    else:
        break


state_fips = states[state_input]

In [ ]:
#use the get() function to request the data from the API
#more information on formatting and access to datasets and variables can be found on the census developers webpage

response = requests.get("https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B19013_001E&for=block group:*&in=state:{}&in=county:*&in=tract:*&key={}".format(state_fips, census_api_key))

In [ ]:
#run this to check if everything is running properly
#here is a good reference guide for all of the possible responses: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status
print(response.status_code)

In [ ]:
#load the data as a json file and print to check if everything looks fine
result = json.loads(response.text)

#print(result)

In [ ]:
#use pandas to convert the json into csv files and save into directory
if os.path.isfile(r'income_bg_{}.csv'.format(state_input)):
    while True:
        file_save_input = input('File already exists. Would you like to overwrite it?(y/n)')
        if file_save_input == 'y' or file_save_input == 'Y':
            print('overwriting file in directory')
            pd.DataFrame(result).to_csv(r'income_bg_{}.csv'.format(state_input))
            break
        elif file_save_input == 'n' or file_save_input == 'N':
            break
        else:
            print('input invalid')
            continue
else:
    print('saving file to directory')
    pd.DataFrame(result).to_csv(r'income_bg_{}.csv'.format(state_input))
#if you want to remove the index from the csv file, simply add ", index = False" after the file directory within the parentheses

In [ ]:
df = pd.read_csv(r'income_bg_{}.csv'.format(state_input))

In [ ]:
#clean the data to how you need it
df.head()

In [ ]:
df.columns

In [ ]:
#change column names
df.columns = ['index', 'NAME', 'GEO_ID', 'INCOME', 'state', 'county', 'tract', 'block_group']
df.head()

In [ ]:
#splitting GEO_ID column to just get the value I need
GEO_ID = df['GEO_ID']
df[['US_Code', 'GEO_ID']] = GEO_ID.str.split("US", n=1, expand=True)

In [ ]:
#delete new column that I don't need
del df['US_Code']
df.head()

In [ ]:
#drop row with same values as column heads
df = df.drop(df.index[0])
df

In [ ]:
#remove nulls
df = df[df.INCOME != '-666666666']
df

In [ ]:
#save final copy

while True:
    save_new = input('Would you like to save a separate copy of your cleaned csv (1), or would you like to overwrite your raw csv (2)? ')
    if save_new == '1':
        os.path.isfile(r'income_bg_{}_clean.csv'.format(state_input)) 
        print('saving file to directory')
        break
    elif save_new == '2':
        os.rename('income_bg_{}.csv'.format(state_input), 'income_bg_{}_clean.csv'.format(state_input))
        pd.DataFrame(df).to_csv(r'income_bg_{}_clean.csv'.format(state_input))
        print('overwriting old file with new file')
        break
    else:
        print('invalid input')
        continue